In [ ]:
# Stylianos Topalidis
# AEM: 9613
# email: styltopa@ece.auth.gr 

In [ ]:
from sklearn import datasets, tree
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

# if the import below does not work please try opening a terminal and writing 
# pip install -U scikit-learn --user
# this worked for me at least in vscode.
# There is some discourse in the forum that it does not work in collab
# see https://stackoverflow.com/questions/72246343/importerror-cannot-import-name-decisionboundarydisplay-from-sklearn-inspecti
from sklearn.inspection import DecisionBoundaryDisplay



In [ ]:
# Load the data
iris = datasets.load_iris()


# First attribute: sepal length (cm)
# Second attribute: sepal width (cm)
irisData = iris.data[:, 0:2]

# Fields of iris data object
# ['DESCR', 'data', 'data_module', 'feature_names', \
#  'filename', 'frame', 'target', 'target_names']

# Percentage of the data for the training 
trainPercent = 0.5


# Turn the target values into a set of all possible targets {0, 1, 2} 
# to exclude duplicates (multiple 0, 1 and 2)
targetSet = set(iris.target)
# Turn the set into a list and then into a numpy array
# for convenience 
targetList = list(targetSet)
targetArr = np.array(targetList) 



# Indexes for all the data samples (iterates over the data 
# for the different targets: setosa, versicolor, virginica) 
indsPerKind = []
# Indexes for the training and testing data in the original dataset
trainingInds = []
testingInds = []


# For each of the flower kinds
for targetCount in range(len(targetArr)):
    indexesArr = np.where(iris.target == targetCount)

    # the first element of the list are the actual data 
    # and the second one is its data type 
    indsPerKind = list(indexesArr[0])

    # number of training data derived from the training percentage selected
    # per group identifier (0, 1, 2)
    numOfTrainingDataPerIdentifier = round(trainPercent*len(indsPerKind))
    numOfTestingDataPerIdentifier = len(indsPerKind) - numOfTrainingDataPerIdentifier

    # concatenate the indices of the new target (identifier) training data 
    # with the indices of the old target training data.
    trainingInds = trainingInds + indsPerKind[0:numOfTrainingDataPerIdentifier]
    testingInds = testingInds + indsPerKind[numOfTrainingDataPerIdentifier:]
    

# list -> np.array
trainingInds = np.array(trainingInds)
testingInds = np.array(testingInds)

# To avoid training the tree with batches of data of the same target,
# as given (all setosa first, then all versicolor and finally all virginica),
# we permute the training data
np.random.seed(0)
trainingIndsPerm =  np.random.permutation(trainingInds)

# Training data and target values
trainingArr = irisData[trainingIndsPerm]
targetArrTraining = iris.target[trainingIndsPerm]

# Testing data and target values
testingArr = irisData[testingInds]
targetArrTesting = iris.target[testingInds]



print('Classification accuracy')
# tree depths
treeDepths = np.array([3, 4, 5])
for depthCount in treeDepths:
    # Classifier training
    clf = tree.DecisionTreeClassifier(max_depth=depthCount)
    clf = clf.fit(trainingArr, targetArrTraining)
    
    # Classifier predictions of the targets
    targetArrPred = clf.predict(testingArr)


    # number of correctly predicted target values
    correctlyPredicted = 0

    numOfTrainingData = len(targetSet)*numOfTrainingDataPerIdentifier


    targetAndPred = np.stack((targetArrTesting, targetArrPred), axis=1) 

    for predCount in range(numOfTrainingData):
        if targetArrTesting[predCount] == targetArrPred[predCount]: 
            # print(targetAndPred[predCount])
            correctlyPredicted = correctlyPredicted + 1
        # else:
            # print(targetAndPred[predCount])


    accuracy = correctlyPredicted/numOfTrainingData
    # print(correctlyPredicted)
    # print(numOfTrainingData)
    print('For decision tree depth = ', depthCount, ': ', round(accuracy*100, 2), '%')
    if depthCount == treeDepths[0]:
        maxAccClf = clf
        maxAcc = accuracy
    if maxAcc < accuracy:
        maxAccClf = clf
        maxAcc = accuracy

print('\nMax accuracy classifier was the one with depth = ', maxAccClf.tree_.max_depth, 'and accuracy: ', round(100*maxAcc, 2), '%')



# do what the cell below does but in a way that you understand it

In [ ]:
# 1) Find the decision bounds from the classifier, make a function to get it from the 
# branches of the tree
# 2) Plot the space partition and their corresponding prdeiction of the target variable.
# 3) Plot the points 



# fig, ax = plt.subplots()
# ax.scatter()
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier


iris = load_iris()
feature_1, feature_2 = np.meshgrid(
    np.linspace(iris.data[:, 0].min(), iris.data[:, 0].max()),
    np.linspace(iris.data[:, 1].min(), iris.data[:, 1].max())
)
grid = np.vstack([feature_1.ravel(), feature_2.ravel()]).T
tree = DecisionTreeClassifier().fit(iris.data[:, :2], iris.target)
y_pred = np.reshape(tree.predict(grid), feature_1.shape)
display = DecisionBoundaryDisplay(
    xx0=feature_1, xx1=feature_2, response=y_pred
)
display.plot()

display.ax_.scatter(
    iris.data[:, 0], iris.data[:, 1], c=iris.target, edgecolor="black"
)

plt.show()
